# Gaussian vs Student’s t Distribution (Single Random Variable)

This notebook showcases how the **Normal (Gaussian)** distribution differs from the **Student’s t** distribution for a single random variable. For basic definitions, see [the wikipedia page](https://en.wikipedia.org/wiki/Student%27s_t-distribution).

We will first compare the shape of the probability density functions (PDFs), their tail behaviour and CDFs (cumulative distribution functions). 

Then, we show how to generate random samples from these distributions, and how to diagnose their difference using Q-Q plots. 

Finally, we show how the Student-t distribution approaches the Gaussian as the number of degrees of freedom increases.


### Analytical Forms of the PDFs

## Gaussian (Normal) Distribution

A Normal random variable with mean $ \mu $ and variance $ \sigma^2 $ has PDF:

$$
f_{\text{Normal}}(x \mid \mu, \sigma^2)
=
\frac{1}{\sqrt{2\pi\sigma^2}}
\exp\!\left(
-\frac{(x-\mu)^2}{2\sigma^2}
\right).
$$

Note that the tails are exponential.

---

## Student’s *t* Distribution

A Student’s *t* random variable with $ \nu $ degrees of freedom has PDF:

$$
f_{t}(x \mid \nu)
=
\frac{\Gamma\!\left(\frac{\nu+1}{2}\right)}
{\sqrt{\nu\pi}\,\Gamma\!\left(\frac{\nu}{2}\right)}
\left(
1 + \frac{x^2}{\nu}
\right)^{-\frac{\nu+1}{2}},
$$

where $ \Gamma(\cdot)$ is the Gamma function, defined as:

$$
\Gamma(z)=\int_{0}^{\infty} t^{z-1} e^{-t}\,dt,
$$

for any $ z \in \mathbb{R}$.

Note the polynomial (power-law) decay of the tail, which are thus heavier than Normal.

As $ \nu \to \infty $, this PDF converges to the Normal $ \mathcal{N}(0,1) $. To obtain it, it suffices to consider the log-PDF, and then taking an easy limit.


## Imports and settings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm, t

np.random.seed(42)

plt.rcParams.update({
    "figure.figsize": (10, 5),
    "axes.grid": True,
    "grid.alpha": 0.3,
})


In [ ]:
x = np.linspace(-6, 6, 2000)

dfs = [2, 5, 10, 30]  # smaller df => heavier tails
normal_pdf = norm.pdf(x, loc=0, scale=1)

plt.figure()
plt.plot(x, normal_pdf, label="Normal(0,1)", linewidth=2, color='black', linestyle='--')

for df in dfs:
    plt.plot(x, t.pdf(x, df=df), label=f"t(df={df})")

plt.title("PDF comparison: Normal vs Student's t")
plt.xlabel("x")
plt.ylabel("density")
plt.legend()
plt.show()


In [ ]:
plt.figure()
plt.semilogy(x, normal_pdf, label="Normal(0,1)", linewidth=2)

for df in dfs:
    plt.semilogy(x, t.pdf(x, df=df), label=f"t(df={df})")

plt.ylim(1e-6, 1)
plt.title("Tail comparison (pdf in log scale)")
plt.xlabel("x")
plt.ylabel("density")
plt.legend()
plt.show()


In [ ]:
plt.figure()
plt.plot(x, norm.cdf(x), label="Normal(0,1)", linewidth=2)
for df in dfs:
    plt.plot(x, t.cdf(x, df=df), label=f"t(df={df})")
plt.title("CDF comparison: Normal vs Student's t")
plt.xlabel("x")
plt.ylabel("probability")
plt.legend()
plt.show()


## Samples generation

In [ ]:
n = 50_000
nu = 3

# Being scipy.stats distributions, we can use their .rvs() method to draw samples. 
# Additionally, since they are vectorized, we can generate all samples in one go.

samples_normal = norm.rvs(size=n)
samples_t      = t.rvs(df=nu, size=n)
samples_t_less = t.rvs(df=nu, size=int(n/10))

bins = np.linspace(-8, 8, 160)

plt.figure()
plt.hist(samples_t, bins=bins, density=True, alpha=0.5, label=f"t(df={nu})")
plt.hist(samples_t_less, bins=bins, density=True, alpha=0.5, label=f"t(df={nu}), less samples")
plt.hist(samples_normal, bins=bins, density=True, alpha=0.5, label="Normal(0,1)")

# overlay PDFs
plt.plot(x, norm.pdf(x), linewidth=2, label="Normal PDF")
plt.plot(x, t.pdf(x, df=nu), linewidth=2, label=f"t(df={nu}) PDF")
plt.title("Samples (histograms) vs theoretical PDFs")
plt.xlabel("x")
plt.ylabel("density")
plt.xlim(-4, 4)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.hist(samples_t, bins=bins, density=True, alpha=0.5, label="t(df=5)")
plt.hist(samples_t_less, bins=bins, density=True, alpha=0.5, label="t(df=5), less samples")
plt.hist(samples_normal, bins=bins, density=True, alpha=0.5, label="Normal(0,1)")

plt.plot(x, norm.pdf(x), linewidth=2, label="Normal PDF")
plt.plot(x, t.pdf(x, df=5), linewidth=2, label="t(df=5) PDF")

plt.yscale("log")
plt.title("Samples with log density scale (highlight tails)")
plt.xlabel("x")
plt.ylabel("density (log scale)")
plt.xlim(-8, 8)
plt.ylim(1e-6, 1)
plt.legend()
plt.show()


In [ ]:
def qq_plot_against_normal(samples, title):

    samples            = np.sort(samples)
    n                  = len(samples)

    # Theoretical quantiles from the normal distribution, see: https://en.wikipedia.org/wiki/Q-Q_plot
    probs              = (np.arange(1, n + 1) - 0.5) / n # Setting the plotting positions of the samples values (arbitrary choice). We shift by 1/2 to avoid probs=0 or 1, for which the quantiles are infinite.
    normal_theoretical = norm.ppf(probs) # The CDF of a gaussian is not known in closed form, hence it is numerically tabulalated, and is known as the error function. Similarly, the inverse CDF (quantile function) is also numerically tabulated, and here accessed via the .ppf() method.

    # For the empirical distribution, we simply use the sorted samples as quantiles. This is because the empirical CDF is a step function that jumps at each sample value, and hence the quantiles correspond to the sorted sample values. Intuitively, the p-th quantile is the value below which p*100% of the data fall, and in a sorted array of n samples, the only way we have to estimate it is to take the i-th smallest sample for the i-th quantile.

    plt.figure()
    plt.scatter(normal_theoretical, samples, s=6, alpha=0.5)
    lo = min(normal_theoretical.min(), samples.min())
    hi = max(normal_theoretical.max(), samples.max())
    plt.plot([lo, hi], [lo, hi], linewidth=2)  # y=x reference
    plt.title(title)
    plt.xlabel("Normal theoretical quantiles")
    plt.ylabel("Sample quantiles")
    plt.show()

    return

qq_plot_against_normal(samples_normal, "QQ plot: Normal samples vs Normal theoretical")
qq_plot_against_normal(samples_t, "QQ plot: t(df=5) samples vs Normal theoretical (tail deviation)")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def plot_df(df=5):

    x = np.linspace(-6, 6, 2000)
    plt.figure(figsize=(10,5))
    plt.plot(x, norm.pdf(x), label="Normal(0,1)", linewidth=3)
    plt.plot(x, t.pdf(x, df=df), label=f"t(df={df})", linewidth=3)
    plt.semilogy()  # comment out to keep linear scale
    plt.ylim(1e-6, 1)
    plt.title("Normal vs t (change df)")
    plt.xlabel("x")
    plt.ylabel("density")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    return

slider = widgets.IntSlider(value=5, min=1, max=100, step=3, description="df")
widgets.interact(plot_df, df=slider)
display(slider)
